# Instalations

In [1]:
# !pip install psgtray
# !pip install pysimplegui
# !pip install keyboard
# !pip install pyautogui

# Imports

In [2]:
from threading import Thread
import PySimpleGUI as sg
from psgtray import SystemTray
import keyboard
from datetime import datetime
import pyautogui
from time import sleep

# Hotkeys

In [3]:
def hotkey_exec(window, i):
    window.write_event_value('-HOTKEY-', f'{i}')
    
def watch_keyboard(window):
    for i in range(0,10):
        keyboard.add_hotkey(f'ctrl + shift + {i}', hotkey_exec, args =(window, i))

def reset_keyboard():
    for i in range(0,10):
            keyboard.remove_hotkey(f'ctrl + shift + {i}')

# Snippets Func

In [4]:
add_snippet = lambda key, text: keyboard.add_abbreviation(key, text)
rm_snippet = lambda key: keyboard.remove_abbreviation(e)

# Actions

In [5]:
def exec_script(script:str):
    pass

def exec_hotkeys(keys:str):
    pass

def exec_system_termina(cmd:str):
    pass

def activate_snippets():
    pass

def deactivate_snippets():
    pass



# Mode

In [6]:
class Mode(object):
    def __init__(self, 
                 name:str
                ):
        self.__name = name

    def __str__(self):
        return self.__name


# GUI

In [7]:
# Gerando elemento de cada botão
def gen_key_comp(i:int):
    i = i + 1 if i < 9 else 0 
    return sg.Frame(
        f'FUNC-{i}',
        [
        [sg.T(f'Flavor text of Func {i}', key=f'-DESCRIPTION{i}-', s=(15,2))],
        [sg.Image(sg.EMOJI_BASE64_HAPPY_THUMBS_UP, size=(100, 100), key=f'-THUMB-{i}')],
        [sg.Button('Edit', key=f'-EDIT{i}-', expand_x=True)]
        ]
    )

# Gerando linhas de botões
gen_key_lines = lambda start, end: [gen_key_comp(i) for i in range(start, end)]
        


def main():

    
    th_detect = {}
    
    # Estado da aplicação
    all_modes = ['Inativo', 'Stream Deck', 'Rodrigo Faro', 'Devil Man']
    cur_mode = 0
    selected_mode = 0
    in_selection = False
    time_last_comand = datetime.now() # Iniciando em uma data específica
    __keep_ths = True
    
    menu = [f'MODO: {cur_mode}', ['Show Window', 'Hide Window', '---', 'Exit']]
    
    layout = [
        gen_key_lines(0,5),
        gen_key_lines(5,10),
        [sg.Button('Exit', expand_x=True)]
    ]
    
    tooltip = lambda x = cur_mode: f'Modo: {x}'

    
    def __create_overlay(
        name:str, layout:list, 
        pos:tuple=(0,0), size:tuple=(400,200)
    ):
        nonlocal th_detect

        __xpos, __ypos = pos
        __xsize, __ysize = size
        __window = sg.Window( name, layout,
            location=(__xpos, __ypos), size=(__xsize, __ysize), 
            keep_on_top=True, auto_size_buttons=False, grab_anywhere=False, return_keyboard_events=False, 
            alpha_channel=0.6, use_default_focus=False, finalize=True, no_titlebar=True )

        def __is_over_window():
            nonlocal name, __xpos, __ypos, __xsize, __ysize, __window, __keep_ths
            while True:
                if not __keep_ths: break
                x, y = pyautogui.position()
                res = (__ypos < y < (__ypos + __ysize)) * (__xpos < x < (__xpos + __xsize))
                __window.write_event_value(f'-SHOW-', f'{res}')
                sleep(0.3) 

        th = Thread(target=__is_over_window)
        th.start()
        th_detect[name] = th

        return __window

    
    def show_message(text:str):
        nonlocal tray 
        tray.show_message('PyHotKeys', text)
        
        
    def mode_navigate(add:bool):
        nonlocal all_modes, cur_mode, selected_mode
        len_modes = len(all_modes)
        
        selected_mode = selected_mode + 1 if add else selected_mode + 1
        selected_mode = 0  if len_modes < selected_mode + 1 else selected_mode
        selected_mode = len_modes - 1  if selected_mode < 0 else selected_mode
        
        
        show_message(f"ATUAL: {all_modes[cur_mode]}\nSELECIONADO: {all_modes[selected_mode]}")
    

    def __exit():
        nonlocal __keep_ths, over1, over2, tray, window
        __keep_ths = False
        over1.close()
        over2.close()
        tray.close()
        reset_keyboard()
        window.close()

        
    #Iniciando telas e observadores
    over1 = __create_overlay('Hotkeys', [[sg.T('HotKeys')]])
    over2 = __create_overlay('Player', [[sg.T('Player')]], pos=(0, 200))
    window = sg.Window('PyHotKeys', layout, finalize=True, enable_close_attempted_event=True)
    watch_keyboard(window)
    tray = SystemTray(menu, single_click_events=False, window=window, tooltip=tooltip(), icon=sg.EMOJI_BASE64_HAPPY_JOY)

    
    #Início do processo
    show_message(f'Iniciado, modo no modo: {cur_mode}')
    
    while True:
        window, event, values = sg.read_all_windows()
        if event in (sg.WIN_CLOSED,  'Cancel'):
            break

        if event == '-SHOW-':
            val = int(values[0])
            if val:
                window.hide()
            else:
                window.un_hide()
                
                
        if event == '-HOTKEY-':
            cur_time = datetime.now()
            diff_last_time = (cur_time - time_last_comand).total_seconds()
            
            # Bloqueando comandos com menos de 1 segundo de tempo
            time_last_comand = cur_time if diff_last_time > 1 else time_last_comand
            if values[event] == '0':
                if in_selection:
                    in_selection = False
                    cur_mode = selected_mode
                    show_message(f"Selecionado modo: {all_modes[cur_mode]}")

                else:
                    in_selection = True
                    show_message(f"Entrando em modo de seleção.\nAtualmente em: {all_modes[cur_mode]}")


            if values[event] == '8':
                if in_selection == True:
                    mode_navigate(False)

            if values[event] == '9':
                if in_selection == True:
                    mode_navigate(True)


            if values[event].isnumeric() and not values[event] in '89':
                if in_selection == False:
                    print('ok')
                        
        
        if event in ('Show Window', sg.EVENT_SYSTEM_TRAY_ICON_DOUBLE_CLICKED):
            window.un_hide()
            window.bring_to_front()
        elif event in ('Hide Window', sg.WIN_CLOSE_ATTEMPTED_EVENT):
            window.hide()
            tray.show_icon()


            
    #Matando processos
    __exit()
        

    
    
    
if __name__ == '__main__':
    main()